In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Install the Libraries
!pip install -q langchain torch transformers sentence-transformers datasets faiss-cpu unstructured chromadb gradio

In [4]:
# Import Libraries
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
import gradio as gr

In [5]:
# Load Documents from Google Drive
loader = DirectoryLoader('/content/drive/MyDrive/preprocessed_fyp')
data = loader.load()

In [6]:
# Split Documents into Chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)
documents = text_splitter.split_documents(data)

In [7]:
# Text Embedding Model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
# Create the Vector Store with Chroma
db = FAISS.from_documents(documents, embeddings)

In [9]:
# Test Similarity Search
question = "What is chemical engineering?"
search_docs = db.similarity_search(question)
print(f"Sample Search Result: {search_docs[0].page_content}")

Sample Search Result: measurable way there may be more than one measurable specific objectives defined for a given competency the specific objectives identified for chemical engineering program are presented below 1 apply knowledge of basic science engineering fundamentals and chemical engineering principles to solve complex engineering problems 2 identify formulate and analyze complex chemical engineering problems to design their solutions develop systems and processes that meet specified needs with appropriate consideration for public health safety and environmental considerations using principles of science and chemical engineering 3 demonstrate knowledge and understanding of engineering management principles and economic decisionmaking and apply them to manage projects in multidisciplinary environments 4 recognize the need for and have the preparation and ability to engage in independent and lifelong learning in the broadest context of technological changes in chemical engineering 

In [10]:
# Prepare the LLM Model (distilGPT2)
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
model = GPT2LMHeadModel.from_pretrained("distilgpt2")


In [17]:
# Create Text Generation Pipeline with `max_new_tokens`
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024,  # Adjusted max_length
    max_new_tokens=200  # Controls the number of tokens generated
)

In [18]:
# Create HuggingFacePipeline LLM
llm = HuggingFacePipeline(pipeline=text_generator)

In [19]:
# Create Retriever
retriever = db.as_retriever(search_kwargs={"k": 2})

In [20]:
# Setup the RetrievalQA Chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=retriever,
    return_source_documents=False
)


In [21]:
# Function to Query the Chatbot
def query_chatbot(question: str) -> str:
    try:
        result = qa.run(question)
        return result
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [22]:
# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# AASTU_Chat.AI")
    question_input = gr.Textbox(label="Ask a Question")
    answer_output = gr.Textbox(label="Answer", interactive=False)
    submit_button = gr.Button("Submit")
    submit_button.click(fn=query_chatbot, inputs=question_input, outputs=answer_output)

# Launch Gradio App
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://942d0e974c23d0c525.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


**With Prompt Template**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# # Install the Libraries
# !pip install -q langchain torch transformers sentence-transformers datasets faiss-cpu unstructured chromadb gradio

# # Import Libraries
# from langchain.document_loaders import DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Chroma
# from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
# from langchain import HuggingFacePipeline, LLMChain
# from langchain.chains import RetrievalQA, StuffDocumentsChain
# from langchain.prompts import PromptTemplate
# import gradio as gr

# # Load Documents from Google Drive
# loader = DirectoryLoader('/content/drive/MyDrive/preprocessed_fyp')
# data = loader.load()

# # Split Documents into Chunks
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200,
#     separators=["\n\n", "\n", ".", " ", ""]
# )
# documents = text_splitter.split_documents(data)

# # Text Embedding Model
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # Create the Vector Store with Chroma
# db = Chroma.from_documents(documents, embeddings)

# # Test Similarity Search
# question = "What is chemical engineering?"
# search_docs = db.similarity_search(question)
# print(f"Sample Search Result: {search_docs[0].page_content}")

# # Prepare the LLM Model (distilGPT2)
# tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
# model = GPT2LMHeadModel.from_pretrained("distilgpt2")

# # Create Text Generation Pipeline with `max_new_tokens`
# text_generator = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_length=1024,  # Adjusted max_length
#     max_new_tokens=200  # Controls the number of tokens generated
# )

# # Create HuggingFacePipeline LLM
# llm = HuggingFacePipeline(pipeline=text_generator)

# # Create Retriever
# retriever = db.as_retriever(search_kwargs={"k": 4})

# # Define the Prompt Template
# prompt_template = """
# Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

# {context}

# Question: {question}
# Helpful Answer:
# """

# prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# # Create the LLM Chain
# llm_chain = LLMChain(
#     llm=llm,
#     prompt=prompt
# )

# # Create the StuffDocumentsChain
# stuff_docs_chain = StuffDocumentsChain(
#     llm_chain=llm_chain
# )

# # Combine into RetrievalQA Chain
# qa = RetrievalQA(
#     retriever=retriever,
#     combine_documents_chain=stuff_docs_chain
# )

# # Function to Query the Chatbot
# def query_chatbot(question: str) -> str:
#     try:
#         result = qa.run(question)
#         return result
#     except Exception as e:
#         return f"An error occurred: {str(e)}"

# # Gradio Interface
# with gr.Blocks() as demo:
#     gr.Markdown("# Retrieval-Augmented Generation (RAG) Chatbot")
#     question_input = gr.Textbox(label="Ask a Question")
#     answer_output = gr.Textbox(label="Answer", interactive=False)
#     submit_button = gr.Button("Submit")
#     submit_button.click(fn=query_chatbot, inputs=question_input, outputs=answer_output)

# # Launch Gradio App
# demo.launch(share=True)

**With Prompt**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# # Install the Libraries
# !pip install -q langchain torch transformers sentence-transformers datasets faiss-cpu unstructured chromadb gradio

# # Import Libraries
# from langchain.document_loaders import DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Chroma
# from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
# from langchain import HuggingFacePipeline
# from langchain.chains import RetrievalQA
# import gradio as gr

# # Load Documents from Google Drive
# loader = DirectoryLoader('/content/drive/MyDrive/preprocessed_fyp')
# data = loader.load()

# # Split Documents into Chunks
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200,
#     separators=["\n\n", "\n", ".", " ", ""]
# )
# documents = text_splitter.split_documents(data)

# # Text Embedding Model
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # Create the Vector Store with Chroma
# db = Chroma.from_documents(documents, embeddings)

# # Test Similarity Search
# question = "What is chemical engineering?"
# search_docs = db.similarity_search(question)
# print(f"Sample Search Result: {search_docs[0].page_content}")

# # Prepare the LLM Model (distilGPT2)
# tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
# model = GPT2LMHeadModel.from_pretrained("distilgpt2")

# # Create Text Generation Pipeline with `max_new_tokens`
# text_generator = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_length=1024,  # Adjusted max_length
#     max_new_tokens=200  # Controls the number of tokens generated
# )

# # Create HuggingFacePipeline LLM
# llm = HuggingFacePipeline(pipeline=text_generator)

# # Create Retriever
# retriever = db.as_retriever(search_kwargs={"k": 4})

# # Setup the RetrievalQA Chain
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=retriever,
#     return_source_documents=False
# )

# # Function to Query the Chatbot
# def query_chatbot(question: str) -> str:
#     try:
#         result = qa.run(question)
#         return result
#     except Exception as e:
#         return f"An error occurred: {str(e)}"

# # Gradio Interface
# with gr.Blocks() as demo:
#     gr.Markdown("# Retrieval-Augmented Generation (RAG) Chatbot")
#     question_input = gr.Textbox(label="Ask a Question")
#     answer_output = gr.Textbox(label="Answer", interactive=False)
#     submit_button = gr.Button("Submit")
#     submit_button.click(fn=query_chatbot, inputs=question_input, outputs=answer_output)

# # Launch Gradio App
# demo.launch(share=True)